# Hands-on example

In [1]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet


In [2]:
import mlflow

## 0. The data

* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)
data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4646,5.9,0.18,0.28,1.0,0.037,24.0,88.0,0.99094,3.29,0.55,10.65,7
3165,6.1,0.68,0.52,1.4,0.037,32.0,123.0,0.99022,3.24,0.45,12.00,6
3693,6.8,0.31,0.30,8.0,0.028,33.0,122.0,0.99164,3.13,0.63,12.60,7
1064,8.1,0.26,0.37,1.9,0.072,48.0,159.0,0.99490,3.37,0.70,10.90,6
2006,8.4,0.20,0.38,11.8,0.055,51.0,170.0,1.00040,3.34,0.82,8.90,6
3485,7.5,0.21,0.32,4.8,0.056,39.0,113.0,0.99393,3.11,0.52,10.20,7
483,6.7,0.47,0.34,8.9,0.043,31.0,172.0,0.99640,3.22,0.60,9.20,5
4617,5.2,0.50,0.18,2.0,0.036,23.0,129.0,0.98949,3.36,0.77,13.40,7
1646,6.4,0.22,0.49,7.5,0.054,42.0,151.0,0.99480,3.27,0.52,10.10,6
710,6.1,0.25,0.24,12.1,0.046,51.0,172.0,0.99800,3.35,0.45,9.50,5


## 1. Tracking experiments

### Tracking stores
MLflow supports two types of backend stores: *file store* and *database-backed* store.

- Local file path (specified as file:/my/local/dir), where data is just directly stored locally. Defaults to `mlruns/`
- Database encoded as <dialect>+<driver>://<username>:<password>@<host>:<port>/<database>. Mlflow supports the dialects mysql, mssql, sqlite, and postgresql. For more details, see SQLAlchemy database uri.
- HTTP server (specified as https://my-server:5000), which is a server hosting an MLFlow tracking server.
- Databricks workspace (specified as databricks or as databricks://<profileName>, a Databricks CLI profile.
    
### Artifact stores
- Amazon S3
- Azure Blob Storage
- Google Cloud Storage
- FTP server
- SFTP Server
- NFS
- HDFS

Start the MLflow tracking server by 

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0
    --port 5000
```

or use the default storage method to write to `mlruns/`.

In [4]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://0.0.0.0:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [5]:
mlflow.tracking.get_tracking_uri()

'http://0.0.0.0:5000'

In [6]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/06/07 23:12:46 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/prithiveer/exercise_3.2/mlruns/856422948774763353', creation_time=1717782166650, experiment_id='856422948774763353', last_update_time=1717782166650, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

### What do we track?

- **Code Version**: Git commit hash used for the run (if it was run from an MLflow Project)
- **Start & End Time**: Start and end time of the run
- **Source**: what code run?
- **Parameters**: Key-value input parameters.
- **Metrics**: Key-value metrics, where the value is numeric (can be updated over the run)
- **Artifacts**: Output files in any format.

In [7]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, "model")

In [12]:
# could also do
# with mlflow.start_run():
#     for epoch in range(0, 3):
#          mlflow.log_metric(key="quality", value=2*epoch, step=epoch)

In [8]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: /home/prithiveer/exercise_3.2/mlruns/856422948774763353/48651712903b404ab450ea312de87eb7/artifacts


In [9]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.20224631822892092
Save to: /home/prithiveer/exercise_3.2/mlruns/856422948774763353/aa9db6718aa74a64a068b3b5ee66464d/artifacts


In [10]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118586
  R2: 0.2157063843066196
Save to: /home/prithiveer/exercise_3.2/mlruns/856422948774763353/b5e7a1a0a61343d89b89f00c17ea6cc2/artifacts


### 1.1 Comparing runs
Run `mlflow ui` in a terminal or `http://your-tracking-server-host:5000` to view the experiment log and visualize and compare different runs and experiments. The logs and the model artifacts are saved in the `mlruns` directory (or where you specified).

## 2. Packaging the experiment as a MLflow project as conda env

Specify the entrypoint for this project by creating a `MLproject` file and adding an conda environment with a `conda.yaml`. You can copy the yaml file from the experiment logs.

To run this project, invoke `mlflow run . -P alpha=0.42`. After running this command, MLflow runs your training code in a new Conda environment with the dependencies specified in `conda.yaml`.


## 3. Deploy the model

Deploy the model locally by running 

`mlflow models serve -m mlruns/0/f5f7c052ddc5469a852aa52c14cabdf1/artifacts/model/ -h 0.0.0.0 -p 1234`

Test the endpoint:

`curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://0.0.0.0:1234/invocations`

You can also simply build a docker image from your model

`mlflow models build-docker -m mlruns/1/d671f37a9c7f478989e67eb4ff4d1dac/artifacts/model/ -n elastic_net_wine`

and run the container with

`docker run -p 8080:8080 elastic_net_wine`.

Or you can directly deploy to AWS sagemaker or Microsoft Azure ML.

## 4. Tagging runs

In [13]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()

print(dir(client))
experiments = client.search_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_artifact_file_string', '_create_model_version', '_get_registry_client', '_log_artifact_async_helper', '_log_artifact_helper', '_read_from_file', '_record_logged_model', '_registry_uri', '_start_tracked_trace', '_tracking_client', '_upload_ended_trace_info', '_upload_trace_data', '_upload_trace_spans_as_tag', 'copy_model_version', 'create_experiment', 'create_model_version', 'create_registered_model', 'create_run', 'delete_experiment', 'delete_model_version', 'delete_model_version_tag', 'delete_registered_model', 'delete_registered_model_alias', 'delete_registered_model_tag', 'delete_run', 'delete_tag', 'delete_trace_tag', '

In [14]:
# get the run
_run = client.get_run(run_id="7c971234beeb438eb6cc4e7c4ea54e2c")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6177463381374035,
 'r2': 0.19042414117747397,
 'rmse': 0.7917149365733894}, params={'alpha': '0.42', 'l1_ratio': '0.1'}, tags={'mlflow.gitRepoURL': 'https://github.com/tsterbak/pydataberlin-2019.git',
 'mlflow.log-model.history': '[{"run_id": "7c971234beeb438eb6cc4e7c4ea54e2c", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-06-07 18:19:21.740514", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.3", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.5.0", '
                             '"se

In [15]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)